In [0]:
# Running PySpark at Google Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession



In [0]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').option("numFeatures","10").load('/content/drive/My Drive/Colab Notebooks/sample_linear_regression_data.txt')
# Try the path without any space

In [19]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='Prediction')

In [0]:
lrmodel = lr.fit(training)

In [22]:
lrmodel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [23]:
lrmodel.intercept

0.14228558260358093

In [0]:
training_summary = lrmodel.summary

In [25]:
training_summary.r2

0.027839179518600154

In [26]:
training_summary.rootMeanSquaredError

10.16309157133015

In [0]:
all_data = spark.read.format('libsvm').option("numFeatures","10").load('/content/drive/My Drive/Colab Notebooks/sample_linear_regression_data.txt')

In [0]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [33]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
+----------

In [34]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                363|
|   mean|0.23630704442854822|
| stddev| 10.055632799685409|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [35]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                138|
|   mean|  0.311027846319763|
| stddev| 11.015882799630226|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_result = correct_model.evaluate(test_data)

In [40]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.075540508745366|
| -25.24803159524898|
| -23.37476717376433|
| -20.45582219506037|
|-20.358338885964223|
| -18.99180013311991|
|-20.537192092331637|
| -18.14406179176423|
|-19.800095241361404|
|-14.901393864842722|
|-18.500916527332816|
|-17.154333998340892|
|-15.462131689313656|
| -16.64955687168152|
|-13.780481492352491|
|-14.688485876348796|
| -12.78058181466493|
|-13.528047401472989|
|-15.162486484320217|
|-12.983908149157998|
+-------------------+
only showing top 20 rows



In [42]:
test_result.rootMeanSquaredError

10.943444520642846

In [0]:
unlabeled_data = test_data.select('features')

In [44]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)

In [46]:
predictions.show()

+--------------------+--------------------+
|            features|          Prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|0.029522470968730993|
|(10,[0,1,2,3,4,5,...|  -1.488175587352741|
|(10,[0,1,2,3,4,5,...|-0.13611691655864017|
|(10,[0,1,2,3,4,5,...|  0.5712614207869467|
|(10,[0,1,2,3,4,5,...|  0.6910202705925038|
|(10,[0,1,2,3,4,5,...| 0.14587766022132678|
|(10,[0,1,2,3,4,5,...|   2.261978526327007|
|(10,[0,1,2,3,4,5,...|  0.3404356030997146|
|(10,[0,1,2,3,4,5,...|  2.3714206704218976|
|(10,[0,1,2,3,4,5,...|  -2.125098399366825|
|(10,[0,1,2,3,4,5,...|  2.7202314947095143|
|(10,[0,1,2,3,4,5,...|   1.422245726101646|
|(10,[0,1,2,3,4,5,...| 0.08627396600135882|
|(10,[0,1,2,3,4,5,...|  1.2900119918488393|
|(10,[0,1,2,3,4,5,...| -1.5683896630267629|
|(10,[0,1,2,3,4,5,...| -0.3679970981936379|
|(10,[0,1,2,3,4,5,...| -1.9821764382661966|
|(10,[0,1,2,3,4,5,...| 0.10745262558223223|
|(10,[0,1,2,3,4,5,...|   2.240263380949797|
|(10,[0,1,2,3,4,5,...|  0.504627